In [11]:
from vnstock import *
import pandas as pd
import numpy as np
import re
import datetime
from functools import reduce
import inspect
# from math import *

In [7]:
stock_history_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock History.csv")
income_statement_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Income Statement.csv")
balance_sheet_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Balance Sheet.csv")
cash_flow_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Cash Flow.csv")
listing_companies_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Listing Companies.csv")
financial_ratio_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Ratio.csv")
stock_intraday_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock Intraday Transaction.csv")

general_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/General Rating.csv")
business_model_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Model Rating.csv")
business_operation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Operation Rating.csv")
financial_health_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Health Rating.csv")
valuation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Valuation Rating.csv")
industry_financial_health_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Industry Financial Health.csv")

# Dag generator

In [17]:
def dag_generator(df, eq, name, period):

    #Imports and default arguments
    script = f"""
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import timedelta
from vnstock import *
import pendulum
from math import *
import pandas as pd
import numpy as np  

default_args = {{
'owner': 'jazzdung',
'retries':5,
'retry_delay': timedelta(minutes=20)
}}


full_indicator = pd.read_csv("/opt/airflow/data/full_indicator.csv")

def equation_calculator(df, eq, name):

    #calculate equation, just to save result on 
    result = df.eval(eq)
    result = result.to_frame()
    result.rename(columns={{0: 'result'}}, inplace=True)
    result.head()
    
    #Write result to csv file (Just for now)
    result.to_csv('/opt/airflow/data/' + '{name}' + '.csv', sep=',', encoding='utf-8')

    return result

with DAG(
    dag_id='cash_flow_v1',
    description='Import to cash flow table',
    start_date=pendulum.yesterday(),
    schedule_interval='period',
    catchup=True
) as dag:

    task1 = PythonOperator(
        task_id = 'equation_calculator',
        python_callable = equation_calculator
        op_kwargs={{
            'df': df,
            'eq': eq
            'name': name}}
    )

    task1

"""

    #Write to py file
    with open("/home/jazzdung/Project III/Airflow/script/python/" + name + ".py", "w+") as dag:
        dag.write(script)

    return script

In [ ]:
def equation_calculator(df, eq, name):

    #calculate equation, just to save result on 
    result = df.eval(eq)
    result = result.to_frame()
    result.rename(columns={0: 'result'}, inplace=True)
    result.head()
    
    #Write result to csv file (Just for now)
    result.to_csv('/home/jazzdung/Project III/Airflow/data/airflow/' + name + '.csv', sep=',', encoding='utf-8')

    return result

In [8]:
#Get list of dataframe
full_indicator_list = [income_statement_table, balance_sheet_table, cash_flow_table, financial_ratio_table]

#Join dataframes
full_indicator = reduce(lambda left, right: pd.merge(left, right, on=['ticker', 'year', 'quarter'], how = 'outer'), full_indicator_list)

# for col in full_indicator.columns:
#     print(col)

full_indicator = full_indicator.set_index(['ticker','year','quarter'])
full_indicator = full_indicator.sort_values(by=['ticker', 'year','quarter'], ascending=[True, False, False])
full_indicator.to_csv('/home/jazzdung/Project III/Airflow/data/airflow/full_indicator.csv', sep=',', encoding='utf-8')

full_indicator

revenue  yearRevenueGrowth  quarterRevenueGrowth  \
ticker year quarter                                                     
A32    2021 5            NaN                NaN                   NaN   
       2020 5            NaN                NaN                   NaN   
       2019 5            NaN                NaN                   NaN   
       2018 5            NaN                NaN                   NaN   
AAA    2022 3         3280.0             -0.037                -0.283   
...                      ...                ...                   ...   
YTC    2008 5          249.0              0.109                   NaN   
       2007 5          225.0              0.318                   NaN   
       2006 5          171.0             -0.197                   NaN   
       2005 5          212.0             -0.652                   NaN   
       2004 5          611.0                NaN                   NaN   

                     costOfGoodSold  grossProfit  operationExpense  \
ticker year quarter                                                  
A32    2021 5                   NaN          NaN               NaN   
       2020 5                   NaN          NaN               NaN   
       2019 5                   NaN          NaN               NaN   
       2018 5                   NaN          NaN               NaN   
AAA    2022 3               -2972.0        308.0            -199.0   
...                             ...          ...               ...   
YTC    2008 5                -226.0         23.0             -17.0   
       2007 5                -209.0         16.0             -12.0   
       2006 5                -157.0         13.0              -9.0   
       2005 5                -200.0         13.0             -11.0   
       2004 5                -579.0         32.0             -17.0   

                     operationProfit  yearOperationProfitGrowth  \
ticker year quarter                                               
A32    2021 5                    NaN                        NaN   
       2020 5                    NaN                        NaN   
       2019 5                    NaN                        NaN   
       2018 5                    NaN                        NaN   
AAA    2022 3                  109.0                      0.501   
...                              ...                        ...   
YTC    2008 5                    6.0                      0.509   
       2007 5                    4.0                     -0.098   
       2006 5                    4.0                      2.073   
       2005 5                    1.0                     -0.902   
       2004 5                   14.0                        NaN   

                     quarterOperationProfitGrowth  interestExpense  ...  \
ticker year quarter                                                 ...   
A32    2021 5                                 NaN              NaN  ...   
       2020 5                                 NaN              NaN  ...   
       2019 5                                 NaN              NaN  ...   
       2018 5                                 NaN              NaN  ...   
AAA    2022 3                               0.042            -40.0  ...   
...                                           ...              ...  ...   
YTC    2008 5                                 NaN            -22.0  ...   
       2007 5                                 NaN             -1.0  ...   
       2006 5                                 NaN              0.0  ...   
       2005 5                                 NaN              0.0  ...   
       2004 5                                 NaN             -1.0  ...   

                     loanOnAsset  loanOnDeposit  depositOnEarnAsset  \
ticker year quarter                                                   
A32    2021 5                NaN            NaN                 NaN   
       2020 5                NaN            NaN                 NaN   
       2019 5               

In [62]:
#Get list of dataframe
full_rating_list = [general_rating_table, business_model_rating_table, business_operation_rating_table, financial_health_rating_table, valuation_rating_table, industry_financial_health_table]

#Join dataframes
full_rating = reduce(lambda left, right: pd.merge(left, right, on=['ticker'], suffixes=('', '_delme'), how = 'outer'), full_rating_list)

#Remove duplicate columns
full_rating = full_rating[[c for c in full_rating.columns if not c.endswith('_delme')]]

# for col in full_rating.columns:
#     print(col)

full_rating = full_rating.sort_values(by=['ticker'], ascending=True)
full_rating.head()

,ticker,stockRating,valuation,financialHealth,businessModel,businessOperation,rsRating,taScore,highestPrice,lowestPrice,...,netDebtEquity,currentRatio,quickRatio,interestCoverage,netDebtEBITDA,pe,pb,ps,evebitda,dividendRate
0,A32,NaN,4.0,2.6,3.5,2.1,NaN,NaN,34031.0,24314.7,...,5.0,3.0,3.0,1.0,1.0,4.0,4.0,5.0,5.0,1.0
1,AAA,2.6,3.6,4.0,3.2,2.6,1.4,1.0,22800.0,5690.0,...,4.0,4.0,4.0,4.0,4.0,3.0,5.0,5.0,3.0,2.0
2,AAM,3.3,3.6,5.0,NaN,1.1,3.4,NaN,17900.0,9290.0,...,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,1.0
3,AAS,2.9,4.0,5.0,2.8,2.9,1.8,1.0,35528.0,7066.0,...,5.0,5.0,5.0,5.0,5.0,5.0,4.0,3.0,5.0,1.0
4,AAT,2.9,4.0,4.8,3.0,2.8,1.5,1.0,18866.3,4800.0,...,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,1.0


In [43]:
dag_generator(balance_sheet_table, 'longAsset - fixedAsset', 'testing', None)

,result
0,29.0
1,22.0
2,10.0
3,379.0
4,27.0
...,...
67108,0.0
67109,NaN
67110,0.0
67111,5.0


In [41]:
dag_generator(full_indicator, 'longAsset - fixedAsset', 'testing', None)

,result
0,1.0
1,209.0
2,2375.0
3,306.0
4,131.0
...,...
48632,0.0
48633,NaN
48634,4.0
48635,0.0


In [9]:
full_indicator = pd.read_csv("/home/jazzdung/Project III/Airflow/data/airflow/full_indicator.csv")

for col in full_indicator.columns:
    print(col)

ticker
year
quarter
revenue
yearRevenueGrowth
quarterRevenueGrowth
costOfGoodSold
grossProfit
operationExpense
operationProfit
yearOperationProfitGrowth
quarterOperationProfitGrowth
interestExpense
preTaxProfit
postTaxProfit
shareHolderIncome
yearShareHolderIncomeGrowth
quarterShareHolderIncomeGrowth
investProfit
serviceProfit
otherProfit
provisionExpense
operationIncome
ebitda
shortAsset
cash
shortInvest
shortReceivable
inventory
longAsset
fixedAsset
asset
debt
shortDebt
longDebt
equity
capital
centralBankDeposit
otherBankDeposit
otherBankLoan
stockInvest
customerLoan
badLoan
provision
netCustomerLoan
otherAsset
otherBankCredit
oweOtherBank
oweCentralBank
valuablePaper
payableInterest
receivableInterest
deposit
otherDebt
fund
unDistributedIncome
minorShareHolderProfit
payable
investCost
fromInvest
fromFinancial
fromSale
freeCashFlow
priceToEarning
priceToBook
valueBeforeEbitda
dividend
roe
roa
daysReceivable
daysInventory
daysPayable
ebitOnInterest
earningPerShare
bookValuePerShare
in

In [13]:
def lmao (a, b, c, d):
    arg_names =  inspect.getargspec(lmao).args
    for name in arg_names:
        print repr(locals()[name]),
    return arg_names[0]

lmao('fuck', 'lmao', 'yeet', None)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2544611581.py, line 4)